In [1]:
import os
import pandas as pd
import mne
import numpy as np
from pytorchNN2 import *
from feature_extraction import *
from sklearn.model_selection import train_test_split

C:\Users\20221750\AppData\Local\Temp\ipykernel_18112\1066863684.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Specify the directory paths
records_csv_path = "EEG data/Zhang & Wamsley 2019/Records.csv"
psg_directory = "EEG data/Zhang & Wamsley 2019/Data/PSG/"

# Read the metadata from the CSV file
rec_df = pd.read_csv(records_csv_path, index_col=0)

# Initialize an empty list to store the mainList arrays for each file
all_main_lists = []

In [3]:

# Loop through all files in the PSG directory
for filename in os.listdir(psg_directory):
    file_path = os.path.join(psg_directory, filename)

    # Read the raw EEG data using MNE
    data = mne.io.read_raw_edf(file_path)
    dream_df = data.get_data()

    # Extract EEG data for a specific channel (e.g., "Fp1-REF")
    channels = data.ch_names
    raw_df = pd.DataFrame(dream_df)
    raw_df.index = channels
    eeg_df = raw_df.loc["Fp1-REF"].copy().values

    # Split the EEG data into epochs
    list_of_epochs = [eeg_df[i:i + 800] for i in range(0, len(eeg_df), 800)][:9]  # Adjust the number of epochs as needed

    # Initialize an empty list to store the mainList arrays for each epoch
    main_lists_for_file = []

    # Loop through each epoch
    for epoch in list_of_epochs:
        ft_transform = FourierTransform(400, 0.5)  # Adjust parameters as needed
        ft_output, freq_bin = ft_transform.transform(epoch, 2)
        y = np.square(ft_output)
        new = y[:50]
        main_list_for_epoch = np.append(new, 73)  # Append the additional value (73) to the end (duration)
        main_lists_for_file.append(main_list_for_epoch)

    # Append the mainList array for the file to the overall list
    all_main_lists.extend(main_lists_for_file)

Extracting EDF parameters from c:\Users\20221750\Documents\MEDAI\EEG data\Zhang & Wamsley 2019\Data\PSG\subject010_Morning.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from c:\Users\20221750\Documents\MEDAI\EEG data\Zhang & Wamsley 2019\Data\PSG\subject010_NREM.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from c:\Users\20221750\Documents\MEDAI\EEG data\Zhang & Wamsley 2019\Data\PSG\subject010_REM.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from c:\Users\20221750\Documents\MEDAI\EEG data\Zhang & Wamsley 2019\Data\PSG\subject010_SO1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from c:\Users\20221750\Documents\MEDAI\EEG data\Zhang & Wamsley 2019\Data\PSG\subject010_SO10.edf...
EDF file detected
Setting channe

In [4]:
# Convert the list of lists to a Pandas DataFrame
df = pd.DataFrame(all_main_lists)

# Print or use the DataFrame as needed
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1.171236e-07,7.788929e-06,1.562104e-06,1.279969e-06,3.426676e-06,3.949851e-06,1.149552e-06,2.102898e-06,4.863842e-08,6.113909e-07,...,2.017014e-08,2.263013e-08,4.119712e-08,2.062079e-08,3.811324e-08,3.047458e-08,3.566926e-08,4.630744e-09,3.145080e-08,73.0
1,7.661991e-07,7.588654e-06,1.168321e-06,7.423655e-06,4.496490e-07,7.647554e-07,5.456584e-07,2.103754e-07,7.859926e-07,5.008384e-07,...,4.593041e-08,1.723530e-08,3.854518e-08,3.893291e-08,2.188520e-08,1.636850e-09,2.403777e-10,1.820919e-08,5.392861e-09,73.0
2,1.010234e-08,1.418866e-06,1.099291e-06,7.901794e-07,9.119441e-10,5.424090e-07,2.379280e-07,9.313410e-08,4.065359e-07,1.211472e-06,...,4.010991e-09,2.617232e-08,2.341284e-08,1.645302e-08,1.923256e-08,1.452587e-08,5.793020e-09,1.510138e-08,4.960728e-09,73.0
3,5.200896e-07,6.902042e-07,4.070018e-07,6.979662e-08,2.220266e-06,1.074332e-06,9.363185e-07,2.034669e-06,1.270533e-07,6.107886e-08,...,3.372039e-08,6.770904e-08,7.212399e-09,7.018662e-08,4.524962e-09,3.785309e-09,6.213843e-09,3.251554e-08,1.306230e-09,73.0
4,1.966808e-08,5.668324e-06,4.026191e-06,2.279065e-05,2.163737e-06,1.234868e-05,4.389876e-06,3.303692e-06,8.811199e-08,1.377891e-06,...,1.817723e-09,2.119980e-08,5.489381e-08,3.041347e-08,2.872013e-09,1.576586e-09,1.654545e-07,3.771281e-08,1.279417e-08,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2767,1.894302e-07,2.279368e-08,4.654587e-08,4.006559e-07,2.154647e-07,7.268584e-08,2.457780e-07,2.042442e-07,8.749582e-08,1.174017e-07,...,2.874328e-09,1.172503e-09,1.854763e-11,2.484860e-09,6.448936e-09,1.194650e-09,1.074491e-09,2.038974e-09,1.415750e-09,73.0
2768,1.563770e-07,1.968625e-07,4.595832e-07,2.029195e-08,1.128372e-07,7.621357e-08,7.435098e-08,7.052065e-09,1.057534e-07,4.922708e-08,...,2.791814e-09,8.962072e-10,5.764764e-09,3.906534e-09,1.654221e-09,8.466898e-09,4.435010e-09,2.519530e-09,2.806148e-09,73.0
2769,4.705370e-08,1.999990e-08,8.690858e-08,4.242392e-09,3.815846e-08,7.445098e-09,1.522215e-08,1.028339e-08,8.403064e-09,2.734712e-08,...,3.055839e-09,9.787590e-10,7.743732e-10,7.005721e-10,9.665443e-10,1.556496e-09,1.999498e-09,3.895047e-10,6.319543e-10,73.0
2770,2.000087e-08,8.950595e-08,1.132521e-07,5.661387e-10,9.684303e-09,1.573996e-08,2.101463e-09,1.012797e-08,1.197319e-08,4.400308e-10,...,8.669087e-10,1.278845e-10,6.425824e-10,2.441697e-09,1.840330e-09,5.466151e-10,3.343527e-11,1.449559e-11,1.580821e-09,73.0


In [5]:
# Preparing which columns will be used in input and which ones will be used in output of NN
X = df.iloc[:, :-1]
y = df.iloc[:, -1:]

In [6]:
# Convert df to numpy arrays
X = X.values
y = y.values

In [7]:
# Split Training and Test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [8]:
# Initialize the object of the model
model = Model()

In [9]:
# Selects criterion for which to evaluate the model, I picked L1loss
criterion = nn.L1Loss()
# How the model learns from data and learning rate which is how long it goes without error going down. Important note to self: The lower the learning rate the longer it will take to train a model
#Parameters are the various connections between layers in the NN
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


In [10]:
# Training the model
# How many times the model goes through all layers
numberOfRuns = 100
# Create a loss variable to see how well the model learns, however useless with current data
lossTrack = []
# Loop amount of times set in numberOfRuns
for i in range(numberOfRuns):
    y_pred = model.forward(X_train)
    # Optional loss things I found in a tutorial, good for tracking though
    loss = criterion(y_pred, y_train)
    lossTrack.append(loss.detach().numpy())
    
    if i % 10 == 0:
        print(f'Epoch: {i} and loss: {loss}')
    
    optimizer.zero_grad()
    optimizer.step()
    

Epoch: 0 and loss: 72.98922729492188
Epoch: 10 and loss: 72.98922729492188
Epoch: 20 and loss: 72.98922729492188
Epoch: 30 and loss: 72.98922729492188
Epoch: 40 and loss: 72.98922729492188
Epoch: 50 and loss: 72.98922729492188
Epoch: 60 and loss: 72.98922729492188
Epoch: 70 and loss: 72.98922729492188
Epoch: 80 and loss: 72.98922729492188
Epoch: 90 and loss: 72.98922729492188


c:\Users\20221750\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([2217, 1])) that is different to the input size (torch.Size([2217, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
